In [ ]:
# keras packages
from tensorflow.keras.applications import ResNet50
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten,BatchNormalization


# sklearn packages
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.model_selection import train_test_split

import random
import numpy as np
import cv2
import os
from imutils import paths
import progressbar

# drawing packages
import matplotlib.pyplot as plt
%matplotlib inline

#basic tool
import pandas as pd
import sys
import io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Preprocessing

In [ ]:
np.random.seed(161)

#load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#normalized
x_train = x_train / 255.0
x_test = x_test / 255.0

#divide training set,validation set,test set
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.20, random_state = 31)

In [ ]:
#construct a non-equilibrium sample
def imbalanceData(__format,X_train,y_train):
   """
   __format：The imbalance want to input
   X_train:Train Set -X
   y_train:Train Set -Y
   """
   train_classes, train_class_counts=np.unique(y_train, return_counts=True)
   nb_train_classes = len(train_classes)
   class_indices = [np.where(y_train == i)[0] for i in range(nb_train_classes)]
   imbal_class_indices = [class_idx[:class_count] for class_idx, class_count in zip(class_indices, __format)]
   imbal_class_indices = np.hstack(imbal_class_indices)
   y = y_train[imbal_class_indices]
   X = x_train[imbal_class_indices]  
   return X,y

In [ ]:
imbal_class_counts=[250,750,1250,1750,2250,2750,3250,3750,4250,5000]
imbal_class_counts=[int(i * 4/5) for i in imbal_class_counts]
x_train,y_train=imbalanceData(imbal_class_counts,x_train,y_train)

In [ ]:
# Y-one hot
y_train_oh = to_categorical(y_train)
y_val_oh = to_categorical(y_val)
y_test_oh = to_categorical(y_test)

## Model Building

In [ ]:
#define the model
model=Sequential()

model.add(Conv2D(32,(3,3),padding='same',input_shape=x_train.shape[1:],activation='relu'))
model.add(Conv2D(32,(3,3),padding='same',input_shape=(32,32,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same',input_shape=(32,32,3),activation='relu'))
model.add(Conv2D(64,(3,3),padding='same',input_shape=(32,32,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 64)        36928     
                                                      

In [ ]:
#train the model
print('-----------Training-----------')
model.fit(x_train,y_train_oh,batch_size=32,epochs=50,verbose=2)

#evaluate the model
print('-----------Testing-------------')
scores=model.evaluate(x_val,y_val_oh,verbose=0)
print('loss:%s,accuracy:%s' % (scores[0],scores[1]))

#save the model
# model.save('cifar10_cnn_model.h5')

-----------Training-----------
Epoch 1/50
632/632 - 7s - loss: 1.9532 - accuracy: 0.2956 - 7s/epoch - 11ms/step
Epoch 2/50
632/632 - 6s - loss: 1.6911 - accuracy: 0.4220 - 6s/epoch - 10ms/step
Epoch 3/50
632/632 - 6s - loss: 1.5702 - accuracy: 0.4626 - 6s/epoch - 10ms/step
Epoch 4/50
632/632 - 6s - loss: 1.4727 - accuracy: 0.4965 - 6s/epoch - 10ms/step
Epoch 5/50
632/632 - 6s - loss: 1.3939 - accuracy: 0.5243 - 6s/epoch - 10ms/step
Epoch 6/50
632/632 - 6s - loss: 1.3331 - accuracy: 0.5445 - 6s/epoch - 10ms/step
Epoch 7/50
632/632 - 6s - loss: 1.2849 - accuracy: 0.5602 - 6s/epoch - 10ms/step
Epoch 8/50
632/632 - 6s - loss: 1.2306 - accuracy: 0.5765 - 6s/epoch - 10ms/step
Epoch 9/50
632/632 - 6s - loss: 1.1860 - accuracy: 0.5890 - 6s/epoch - 10ms/step
Epoch 10/50
632/632 - 6s - loss: 1.1408 - accuracy: 0.6069 - 6s/epoch - 10ms/step
Epoch 11/50
632/632 - 6s - loss: 1.0980 - accuracy: 0.6178 - 6s/epoch - 10ms/step
Epoch 12/50
632/632 - 6s - loss: 1.0645 - accuracy: 0.6295 - 6s/epoch - 10ms

In [ ]:
#the final effect of the model on the test set
preds = model.predict(x_test, batch_size=32)
print(classification_report(y_test_oh.argmax(axis=1),preds.argmax(axis=1), digits=4))

              precision    recall  f1-score   support

           0     0.9096    0.1710    0.2879      1000
           1     0.9002    0.3970    0.5510      1000
           2     0.6230    0.4280    0.5074      1000
           3     0.4961    0.3860    0.4342      1000
           4     0.6134    0.4760    0.5360      1000
           5     0.4776    0.6820    0.5618      1000
           6     0.7269    0.7720    0.7488      1000
           7     0.5753    0.8290    0.6792      1000
           8     0.5635    0.9010    0.6933      1000
           9     0.5406    0.8650    0.6654      1000

    accuracy                         0.5907     10000
   macro avg     0.6426    0.5907    0.5665     10000
weighted avg     0.6426    0.5907    0.5665     10000

